# Datos oceanográficos de boyas perfiladoras de la Red Argo con `argopy`

Hackatón de OceanHackWeek en Español 2025. 2025-11-26

En este tutorial exploramos los datos de las boyas perfiladoras que forman parte de la [red global Argo](https://argo.ucsd.edu), utilizando el paquete `argopy` en Python. Incluye la definición de una búsqueda de datos existentes, descarga de los datos y transformación a `Dataset` de `xarray`, creación de gráficas con funciones de `argopy` y funciones más básicas de `xarray`, `matplotlib` y `cartopy`, y algunos procesamientos y análisis enfocados en la Profundidad de la Capa de Mezcla ("Mixed Layer Depth", MLD). Todo, usando como ejemplo motivador la zona de afloramiento del Chorro de Papagayo en la costa Pacífico de Nicaragua y Costa Rica.

**Fuentes de información y entrenamiento adicional sobre `argopy` y la red Argo**

Gran parte del material para este tutorial fue adaptado de estas fuentes, que ofrecen guías y tutoriales extensos:

- Sitio web de red Argo: https://argo.ucsd.edu
- `argopy`: https://argopy.readthedocs.io
- Entrenamiento con `argopy`: https://github.com/euroargodev/argopy-training
- Argo Online School: https://www.euro-argo.eu/argo-online-school/intro.html

## Boyas perfiladoras y Red Argo

Una colaboración internacional entre naciones y entidades coordinadas nacionalmente.

![Boya perfiladora, esquematica](graficas/soloII_complete_with_antenna.jpg)

Fuente: https://argo.ucsd.edu/how-do-floats-work/

![Boya perfiladora, foto](graficas/sci_argo_float.jpg)

Fuente: https://salinity.oceansciences.org/science.htm

**Ciclo de perfilación, Argo**

![Ciclo de perfilación, Argo](graficas/ArgoCycle_Espanol-1536x805.png)

Fuente: https://www.argoespana.es

**Mapa de boyas activas, 2025-11-25**

![boyas activas, 2025-11-25](graficas/Mapa-RedArgo-2025nov25.gif)

Fuente: https://argo.ucsd.edu

## Motivadores personales

### Mi trabajo con datos de boyas perfiladoras, *no* integradas a red Argo

**SQUID EM-APEX Data Browser**

![SQUID EM-APEX Data Browser](graficas/app_screenshot-squid.png)

Fuente: Emilio Mayorga, https://squid-test1.azurewebsites.net

**FlowPilot Visualization Application**

![FlowPilot Visualization Application](graficas/app_screenshot-flowpilot_viz-app-20250719.png)

Fuente: Emilio Mayorga

### Afloramiento por vientos alisios de chorro en América Central

Afloramiento (surgencias) por efecto de vientos de chorro en los golfos de Tehuantepec, Papagayo y Panamá. **Enfatizando el Chorro de Papagayo, Nicaragua - Costa Rica**. Fenómenos de afloramiento con dinámicas diferentes de los afloramientos más paradigmáticos, como en la costa del Perú.

**Foto de parque eólico en el departamento de Rivas, Nicaragua, en las costas del Lago de Nicaragua. 2023-12**

![foto 2023-12](graficas/foto-emilio-2023-dic-parqueeolico-rivas.jpg)

Fuente: Emilio Mayorga

**Clorofila-a superficial por el satélite SeaWiFS, 2001-01-30**

![clorofila seawifs enero](graficas/NASAOceanColor-ThePapagayoWind-Chl.png)

Fuente: https://oceancolor.gsfc.nasa.gov/outreach/ocsciencefocus/ThePapagayoWind.pdf

**SST por el satélite AVHRR, 1996-03**

![sst AVHRR marzo](graficas/NASAOceanColor-ThePapagayoWind-SST.png)

Fuente: https://oceancolor.gsfc.nasa.gov/outreach/ocsciencefocus/ThePapagayoWind.pdf

**Falla en Afloramiento. Por primera vez en los registros, las aguas frías y ricas en nutrientes del Golfo de Panamá no surgieron durante la estación seca**

Instituto Smithsonian de Investigaciones Tropicales (STRI), 2025-09-01. https://stri.si.edu/es/noticia/falla-en-afloramiento

"Los científicos del STRI han estudiado este fenómeno y sus registros indican que esta surgencia estacional, que ocurre entre enero y abril, ha sido una característica constante y predecible del golfo durante al menos 40 años. Sin embargo, los científicos recientemente registraron que en 2025, este proceso oceanográfico vital no ocurrió por primera vez. Como resultado, se atenuaron los descensos de temperatura y los aumentos de productividad típicos de esta época del año."

## Importar librerías

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cmocean  # Mapas de colores disenado para variables oceanograficas
import matplotlib.pyplot as plt
import numpy as np

# Funciones de argopy que usaremos directamente
from argopy import DataFetcher
from argopy.plot import scatter_map, scatter_plot

## Seleccionar y descargar datos de Argo por "región"

Utilizaremos la función (`clase`) [DataFetcher (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher) para definir el dominio (extensión) espacio-temporal de la búsqueda y crear variable (objeto) `cargador`. El dominio define la extensión en latitud, longitud, profundidad, y tiempo.

(Aqui he utilizado el sitio http://bboxfinder.com para ayudarme a definir el rectángulo delimitador de latidudes y longitudes para la región del Golfo de Papagayo: http://bboxfinder.com/#8,-91,12,-86)

`DataFetcher` ofrece las siguientes opciones, que no exploraremos aqui sino que usaremos los valores por defecto:
- modo de usuario (`mode`). Niveles de post-procesamiento: "expert", "standard" (defecto), "research"
- proveedor de datos (`src`). Fuente donde obtener los datos, incluyendo cache local
- datasets (`ds`). "phy", parametros fisicos (defecto), o "bgc", parametros biogeoquimicos de Argo BGC

In [ ]:
%%time

cargador = DataFetcher(cache=True, parallel=True).region([
    -91, -86,  # longitud
    8, 12,     # latitud
    0, 700,    # profundidad. Para reducir el tamano de la descarga, no pedimos el perfil completo
    '2005-01-01', '2025-11-30'  # fechas / tiempo (la fecha maxima es exclusiva)
])

Ese paso fue muy rapido porque aún no hemos descargado los datos. Sólo definimos los parámetros de la búsqueda, con algunas decisiones (valores) explícitas y otras implícitas (por defecto). 

Ahora descargamos los datos con el método `.data`, **convirtendolos a un `Dataset` de `xarray` organizados como "puntos".** Al ejecutar este metodo, el `Dataset` no solo es copiada a la variable `punto_ds`, tambien es almazaneda internamente en el objecto `cargardor`:

In [ ]:
%%time

puntos_ds = cargador.data

puntos_ds.argo

El metodo `.argo` da un resumen de la coleccion de observaciones en puntos que hemos descargado: 812 perfiles conteniendo 241,455 puntos en hasta 551 niveles de profundidad.

### Explorar los datos de puntos

Ahora exploremos el `Dataset` de xarray para ver mas detalles. El `Dataset` tiene una sola dimension, `N_POINTS`, los puntos; variables de temperatura, salinidad, presion, de evaluacion de control de calidad, e informacion sobre la boya y la fase en el ciclo de perfilacion. Tambien contiene metadatos sobre como y cuando se descargaron los datos, y el procesamiento anterior de los datos.

In [ ]:
puntos_ds

Objetos de [DataFetcher (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.fetchers.ArgoDataFetcher.html#argopy.fetchers.ArgoDataFetcher) proveen varias capacidades utiles, incluyendo graficas de uso comun. Veamos el dominio, luego un mapa de los puntos:

In [ ]:
cargador.domain

En este mapa, cada boya es representada por un color, con su secuencia de puntos conectadas con lineas.

In [ ]:
cargador.plot('trajectory', markersize=20, legend=False, padding=[1.0, 0.5]);

Como es un `Dataset` de xarray, tambien podemos hacer graficas directamente con xarray. Aqui vemos un histograma de las profundidades:

In [ ]:
puntos_ds['PRES'].plot.hist(figsize=(6,2));

`DataFetcher` tambien nos permite hacer graficas de una propiedad contra otra. Aqui, un plot convencional oceanografico de profundidad en funcion de temperatura y salinidad.

In [ ]:
cargador.plot('PRES', this_x='TEMP', this_y='PSAL');

Podemos mejorar ese plot con una barra de color, dimensionas mas cuadradas, y otros cambios. Vamos a usar un mapa de colores del paquete `cmocean` que contiene colores personalizados para aplicaciones oceanograficas.

In [ ]:
cargador.plot(
    'PRES', this_x='TEMP', this_y='PSAL',
    vmin=0, vmax=700, cmap=cmocean.cm.deep, cbar=True, figsize=(9, 6)
);

## Transformar los datos a una organizacion por perfiles

Podemos transformar (reoganizar) los datos de coleccion de puntos a una estructura de coleccion de perfiles por profundidad, con el metodo `argo.point2profile()` que es parte de `argopy`. Luego veamos las diferencias en organizacion de los datos.

In [ ]:
perfiles_ds = puntos_ds.argo.point2profile()

perfiles_ds.argo

Nota: Ahora hay mas puntos (`N_POINTS`) que antes. Todavia no se por que ....

In [ ]:
perfiles_ds

Podemos ver que ahora las dimensiones son los perfiles (`N_PROF`) y los niveles de profundidad (`N_LEVELS`).

### Calcular densidad, σ<sub>0</sub>

Calculemos la densidad, como la anomalia de densidad potencial, σ<sub>0</sub>. El calculo esta integrado en `argopy`, y la nueva variable 'SIG0' es anadida a `perfiles_ds` como una nueva `DataArray`:

In [ ]:
perfiles_ds.argo.teos10(['SIG0'])

### Generar plots de perfiles oceanograficos

Con la organizacion de datos por perfil, podemos explorar mas intituitivamente la estructura de la columna de agua. Primero enfoquemonos en un solo perfil de una boya, escogido al azar (`N_PROF` con indice 500). Para esto vamos a combinar la funcionalidad de Matplotlib y de xarray, usando la presion ("PRES") como la variable en el eje vertical (y) y la variable de interes en el eje horizontal, y `yincrease=False` para que los valores de profundidad aumenten hacia abajo:

In [ ]:
_, ax = plt.subplots(ncols=3, figsize=(8, 4), sharey=True)

dsp_perf = perfiles_ds.set_coords("PRES").isel(N_PROF=500)

dsp_perf['TEMP'].plot(ax=ax[0], y='PRES', yincrease=False, color='red')
ax[0].set_title(None)
dsp_perf['PSAL'].plot(ax=ax[1], y='PRES', yincrease=False, color='blue')
dsp_perf['SIG0'].plot(ax=ax[2], y='PRES', yincrease=False, color='black')
ax[2].set_title(None)

plt.suptitle(f"{dsp_perf.TIME.data.astype('datetime64[s]').item():%Y-%m-%d %H:%M}Z");

### Agregacion de perfiles por mes, independiente del ano

Un solo perfil de una sola boya no es muy representativo! Tenemos datos de un area amplia a lo largo de 20 anos. Ignorando la variabilidad interanual, vamos a contrastar la estructura vertical en enero, el mes con mayor afloramiento, y junio, un mes con afloramiento debil.

Primero confirmemos que tenemos una buena distribucion de meses:

In [ ]:
perfiles_ds['TIME.month'].plot.hist(figsize=(8, 1.5));

Ahora calculemos un nuevo `Dataset` donde cada variable es convertida a su promedio de todos los perfiles que ocurrieron en ese mes. Para hacer utilizamos la funcionalidad `groupby()` de xarray (ver el [tutorial sobre datos temporales en Python del Taller Intermedio](https://github.com/Intercoonecta/Talleres_intermedios/blob/ohwe25/6-Octubre-2025/datos_temporales_python/Intro_datos_temporales.ipynb)). Este procedimiento genera una nueva variable y dimension, `month` (mes; el nombre es asignado automaticamente). Ahora las dimensiones del `Dataset` son `month` y `N_LEVELS` (indices de profundidad):

In [ ]:
perfiles_mensuales_ds = perfiles_ds.groupby("TIME.month").mean()
# Le anadimos un atributo 'long_name' a la nueva variable 'month',
# para hacerlo mas entendible y util
perfiles_mensuales_ds['month'].attrs = {'long_name': 'Mes'}

perfiles_mensuales_ds

In [ ]:
_, ax = plt.subplots(ncols=3, sharey=True, figsize=(9, 5))

# Tenemos que reasignar PRES como una coordenada. Y luego retenemos solamenbte enero y junio
perfiles_mensuales_sel_ds = perfiles_mensuales_ds.set_coords("PRES").sel(month=[1, 6])

# Este dictionario es un truco Pythonico para no tener que repetir
# todos estos argumentos en las funciones plot.scatter(), abajo
kwargs = dict(
    y='PRES', yincrease=False, 
    hue='month', marker='+', levels=2, s=10, alpha=1, 
    cmap='Accent',
    add_colorbar=False
)
perfiles_mensuales_sel_ds.plot.scatter(ax=ax[0], x='TEMP', **kwargs)
ax[0].set_title(None)
perfiles_mensuales_sel_ds.plot.scatter(ax=ax[1], x='PSAL', add_legend=True, **kwargs)
ax[1].set_ylabel(None); ax[1].set_title(None)
perfiles_mensuales_sel_ds.plot.scatter(ax=ax[2], x='SIG0', **kwargs)
ax[2].set_ylabel(None); ax[2].set_title(None)

plt.suptitle("Meses: Enero/1 (afloramiento) vs Junio/6")
plt.tight_layout();

## Profundidad de Capa de Mezcla (MLD) por perfil

`argopy` nos permite aplicar funciones especializadas a cada perfil. Usamos esta funcionalidad para calcular la Profundidad de Capa de Mezcla (MLD) por perfil

### Calculos

In [ ]:
def diag_mld(pres, sig0, threshold_depth=10):
    """
    Calcular y regresar un array de numpy con la Profundidad de Capa de Mezcla,
    (Mixed Layer Depth, MLD), usando el metodo Boyer Montégut method 
    con umbral de σ(threshold_depth m) + 0.03 kg.m-3

    threshold_depth (profundidad de umbral) es en metros

    Tomado directamente de un notebook de entrenamiento de argopy,
    https://github.com/euroargodev/argopy-training/blob/main/content/argo-data-manipulation/compute-custom.ipynb
    """
    # Value de umbral de referencia
    threshold = 0.03
    
    # Eliminar valores NaN
    idx = ~np.logical_or(np.isnan(pres), np.isnan(sig0))
    sig0_depth, sig0 = pres[idx], sig0[idx]

    # Chequear si hay puntos validos cerca de la profundidad de referencia (umbral)
    if not np.any((sig0_depth >= 0) & (sig0_depth <= threshold_depth)):
        return np.nan

    # Obtener la densidad de referencia en la profundidad del umbral (threshold_depth)
    index_threshold = np.argmin(np.abs(sig0_depth - threshold_depth))
    sig0_at_threshold = sig0[index_threshold]

    # Encontrar la primera profundidad donde la densidad excede el umbral
    exceeds_threshold = sig0[index_threshold:] > sig0_at_threshold + threshold
    if not np.any(exceeds_threshold):
        return np.nan

    mld_index = np.where(exceeds_threshold)[0][0] + index_threshold
    
    return sig0_depth[mld_index]

`argopy` y `xarray` manejan las manipulaciones de ejes y dimensiones, lo que nos permite enfocarnos en escribir una funcion "reductora" con arrays de 1D por cada parametro en la funcion. En este caso, presion y densidad. Como resultado, anadimos una nueva variable (`DataArray`) "MLD" al `Dataset` `perfiles_ds`.

He escogido **80 metros como la profundidad de umbral.** Normalmente seria 10m, pero en esta region, los resultados no son los que esperaba con 10m. Aun con 80m, **creo que la metodologia de MLD quizas no es tan aplicable aqui!**

In [ ]:
perfiles_ds['MLD'] = perfiles_ds.argo.reduce_profile(
    diag_mld, params=['PRES', 'SIG0'], threshold_depth=80
)

# Le anadimos atributos a la nueva variable 'MLD' para hacerla mas entendible y clara
perfiles_ds['MLD'].attrs = {
    'long_name': 'Profundidad de Capa de Mezcla (MLD)',
    'units': 'db',
    'method': 'Umbral de densidad'
}

perfiles_ds

### Graficas de MLD

#### Distribucion de valores de MLD

In [ ]:
perfiles_ds['MLD'].plot.hist(figsize=(6,2));

#### Perfiles y MLD en grafica integrada

MLD en todos los perfiles integrados en un solo plot de densidad vs profundidad, con el tiempo. Aqui usamos la funcion `scatter_plot` de `argopy`.

In [ ]:
fig, ax, _, _ = scatter_plot(perfiles_ds, 'SIG0', cmap=cmocean.cm.dense, s=12, cbar=True);

ax.plot(perfiles_ds['TIME'], perfiles_ds['MLD'], 'k', label=perfiles_ds['MLD'].attrs['long_name'])

ax.legend(loc='upper left');

De comparacion, la temperatura:

In [ ]:
scatter_plot(perfiles_ds, 'TEMP', cmap=cmocean.cm.thermal, s=12, cbar=True);

#### Mapas de MLD

Usando la funcion `scatter_map` de `argopy`:

In [ ]:
scatter_map(perfiles_ds.isel(N_LEVELS=0), hue='MLD', cmap='Spectral_r', traj=False, legend=False, cbar=True);

Usando `xarray`, `Matplotlib` y `cartopy` nos da mas control, pero con mas pasos.

In [ ]:
_, ax = plt.subplots(ncols=1, subplot_kw=dict(projection=ccrs.PlateCarree()))

perfiles_ds.plot.scatter(
    x='LONGITUDE', y='LATITUDE', hue='MLD', add_colorbar=True,
    transform=ccrs.PlateCarree(),
    ax=ax
)
ax.add_feature(cfeature.LAND, facecolor='gray')
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
perfiles_ds['month'] = perfiles_ds['TIME.month']
perfiles_ds = perfiles_ds.set_coords("month").set_xindex('month')

perfiles_ds

In [ ]:
_, axes = plt.subplots(
    ncols=2, sharey=True, sharex=True, figsize=(12, 5),
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

# Enero
perfiles_ds.sel(month=1).plot.scatter(
    x='LONGITUDE', y='LATITUDE', hue='MLD', add_colorbar=True,
    transform=ccrs.PlateCarree(),
    ax=axes[0]
)
axes[0].add_feature(cfeature.LAND, facecolor='gray')
axes[0].coastlines()
gl = axes[0].gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.top_labels = False
gl.right_labels = False
axes[0].set_title("Enero")

# Junio
perfiles_ds.sel(month=6).plot.scatter(
    x='LONGITUDE', y='LATITUDE', hue='MLD', add_colorbar=True,
    transform=ccrs.PlateCarree(),
    ax=axes[1]
)
axes[1].add_feature(cfeature.LAND, facecolor='gray')
axes[1].coastlines()
gl = axes[1].gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.top_labels = False
gl.right_labels = False
axes[1].set_title("Junio")

plt.tight_layout();

Podriamos generar mapas mas simples pero menos intuitivos con solo `xarray` y `Matplotlib`:

```python
# Un mapa simple con plot.scatter de xarray, sin usar cartopy
perfiles_ds.plot.scatter(x='LONGITUDE', y='LATITUDE', hue='MLD', add_colorbar=True);

# O con Matplotlib directamente:
plt.scatter(x=perfiles_ds.LONGITUDE, y=perfiles_ds.LATITUDE, c=perfiles_ds.MLD, s=10);
```

## Incluir un dashboard en linea aqui mismo, con `argopy`

`argopy` integra una capacidad de generar un "dashboard" (panel de exploracion) interactivo aqui mismo, de un sitio en linea de exploracion de datos Argo. Generaremos uno para una boya especifica, basado en su numero de plataforma.

In [ ]:
np.unique(puntos_ds.PLATFORM_NUMBER.data)

In [ ]:
# Esta es una boya Argo BGC activa lanzada este año, por mi universidad, la Universidad de Washington
plataforma_num = 7901109

In [ ]:
DataFetcher().float(plataforma_num).dashboard()

Hay un par de tipos de dashboards mas accesibles a traves de `argopy` 

## Quería incluir algo sobre datos de oxígeno y clorofila de Argo BGC ...

Pero ya me quedé sin tiempo 😢